In [3]:
import os
import sys

import numpy as np
import pandas as pd

import torch.nn.functional as F

In [22]:
def do_kaggle_metric(predict, truth, threshold=0.5):

    N = len(predict)
    predict = predict.reshape(N, -1)
    truth   = truth.reshape(N, -1)

    predict = predict > threshold
    truth   = truth > 0.5
    intersection = truth & predict
    union        = truth | predict
    iou = intersection.sum(1) / (union.sum(1) + 1e-8)

    #-------------------------------------------
    result = []
    precision = []
    is_empty_truth   = (truth.sum(1) == 0)
    is_empty_predict = (predict.sum(1) == 0)

    threshold = np.array([0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95])
    for t in threshold:
        p = iou >= t

        tp  = (~is_empty_truth)  & (~is_empty_predict) & (iou > t)
        fp  = (~is_empty_truth)  & (~is_empty_predict) & (iou <= t)
        fn  = (~is_empty_truth)  & ( is_empty_predict)
        fp_empty = ( is_empty_truth)  & (~is_empty_predict)
        tn_empty = ( is_empty_truth)  & ( is_empty_predict)

        p = (tp + tn_empty) / (tp + tn_empty + fp + fp_empty + fn)

        result.append( np.column_stack((tp,fp,fn,tn_empty,fp_empty)) )
        precision.append(p)

    result = np.array(result).transpose(1, 2, 0)
    precision = np.column_stack(precision)
    precision = precision.mean(1)

    return precision, result, threshold


In [18]:
predicts = []
truths = []
for i in range(4):
    predicts.append(np.ones((256, 1600, 4)))
    truths.append(np.zeros((256, 1600, 4)))

In [19]:
predicts[0].shape

(256, 1600, 4)

In [20]:
predicts = np.concatenate(predicts).squeeze()
truths = np.concatenate(truths).squeeze()

In [21]:
predicts.shape

(1024, 1600, 4)

In [23]:
precision, _, _ = do_kaggle_metric(predicts, truths, 0.5)
precision = precision.mean()

In [24]:
precision

0.0